<a href="https://colab.research.google.com/github/fpabi/Tareas/blob/main/Tarea_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarea 18
##Fabiana Pavon Arce
##Prof. Lazaro Bustio
###APRENDIZAJE NO SUPERVISADO – AGRUPAMIENTO DE DATOS
Objetivo: Eres un analista de datos en una empresa de marketing y se te ha proporcionado un archivo CSV llamado “datos_clientes.csv” que contiene información sobre diferentes clientes.


In [1]:
import pandas as pd


Cargar los datos desde el csv

In [2]:
csv_path ="/content/datos_clientes.csv"
clientes = pd.read_csv(csv_path)

Estadistica descriptiva de los datos del data frame

In [4]:
clientes.describe()

,Edad,Ingresos,Gasto,Puntuacion
count,483.000000,540.000000,540.000000,546.000000
mean,44.304348,29.184647,0.290031,56.780220
std,19.730459,22.064182,0.221071,45.537355
min,18.000000,1.559945,0.013863,1.000000
25%,30.000000,12.764645,0.136138,22.000000
50%,42.000000,26.888401,0.233527,49.500000
75%,56.000000,38.969482,0.403827,81.000000
max,100.000000,100.000000,1.000000,200.000000
